# Image to Video Generation

This notebook will help you quickly start generating your own videos from images using Amazon Nova Reel.

## Prerequisites

Be sure you've followed the instructions in the [00_initial_setup.ipynb](../00_initial_setup.ipynb) notebook to get things set up.


## Configure credentials and shared parameters

Run the cell below to set the default session configuration for all AWS SDK calls made by the other cells in this notebook. As written, the code will default to using the user credentials you have set as your "default" via the AWS CLI. If you'd like to use different credentials, you can modify the code below to add `aws_access_key_id` and `aws_secret_access_key` arguments to the setup function.


In [ ]:
import boto3
import logging

logging.basicConfig(level=logging.INFO, format='[%(levelname)s] %(message)s')
logger = logging.getLogger(__name__)

MODEL_ID = "amazon.nova-reel-v1:0"

# Set default region and credentials.
boto3.setup_default_session(
    region_name="us-east-1"
)

logger.info("AWS SDK session defaults have been set.")

## Invoking the model

Generating a video takes some time - approximately 3.5 minutes to produce a 6 second video. To accomodate this execution time, the Bedrock Runtime introduces a new asynchronous invocation API. Calling `start_async_invoke()` creates a new invocation job. When the job completes, Bedrock automatically saves the generated video to an S3 bucket you specify.

### Image-to-Video

You can also generate videos by providing an initial starting image and a text prompt. For best results, the text prompt should describe the image and also provide details about the desired action and camera movement you'd like the video to have. Modify the `s3_destination_bucket`, `input_image_path`, and `video_prompt` variables at the start of the code below and then run the cell to start generating your video.



<div style="display: flex; justify-content: space-between;">
    <div style="width: 48%;">
        <p align="center">
            <img src="../images/sample-frame-1.png" width="100%" style="padding: 4px">
            <br>
            <em>Input image</em>
        </p>
    </div>
    <div style="width: 48%">
        <p align="center">
            <video alt="example_text_to_video" controls style="padding: 4px" >
                <source src="../videos/example_image_to_video.mp4" type="video/mp4" >
            </video>
            <br>
            <em>Output video</em>
        </p>
    </div>
</div>

In [ ]:
import json
import random
import base64
import amazon_video_util

"""
IMPORTANT: Modify the S3 destination (s3_destination_bucket) and video prompt (video_prompt) below.
"""

# Specify an S3 bucket for the video output.
s3_destination_bucket = "nova-videos"  # Change this to a unique bucket name.

# Define the image to use as the input image.
input_image_path = "../images/sample-frame-1.png"  # Must be 1280 x 720

# Specify your video generation prompt. Phrase your prompt as a summary rather than a command. Maximum 512 characters.
video_prompt = "dolly forward"

"""
STOP: You should not have to modify anything below this line.
"""

# Set up the S3 client.
s3_client = boto3.client("s3")

# Create the S3 bucket.
s3_client.create_bucket(Bucket=s3_destination_bucket)

# Create the Bedrock Runtime client.
bedrock_runtime = boto3.client("bedrock-runtime")

# Load the input image as a Base64 string.
with open(input_image_path, "rb") as f:
    input_image_bytes = f.read()
    input_image_base64 = base64.b64encode(input_image_bytes).decode("utf-8")

model_input = {
    "taskType": "TEXT_VIDEO",
    "textToVideoParams": {
        "text": video_prompt,
        "images": [
            {
                "format": "png",  # May be "png" or "jpeg"
                "source": {
                    "bytes": input_image_base64
                }
            }
        ]
        },
    "videoGenerationConfig": {
        "durationSeconds": 6,  # 6 is the only supported value currently.
        "fps": 24,  # 24 is the only supported value currently.
        "dimension": "1280x720",  # "1280x720" is the only supported value currently.
        "seed": random.randint(
            0, 2147483648
        ),  # A random seed guarantees we'll get a different result each time this code runs.
    },
}

try:
    # Start the asynchronous video generation job.
    invocation = bedrock_runtime.start_async_invoke(
        modelId="amazon.nova-reel-v1:0",
        modelInput=model_input,
        outputDataConfig={"s3OutputDataConfig": {"s3Uri": f"s3://{s3_destination_bucket}"}},
    )

    # This will be used by other cells in this notebook.
    invocation_arn = invocation["invocationArn"]

    # Pretty print the response JSON.
    logger.info("\nResponse:")
    logger.info(json.dumps(invocation, indent=2, default=str))

    # Save the invocation details for later reference. Helpful for debugging and reporting feedback.
    amazon_video_util.save_invocation_info(invocation, model_input)

except Exception as e:
    logger.error(e)

## Checking the status of generation jobs

We've provided a set of utility functions in the `amazon_video_util.py` script. One of these functions will automatically download a job if it is completed or monitor it while it is in-progress. The `invocation_arn` is defined in the code cell above and passed in below. This function will return the local file path for the generated video for successfully completed jobs.



In [ ]:
local_file_path = amazon_video_util.monitor_and_download_video(invocation_arn, "output")

## Viewing output video
You can then view your output video by running the below cell:

In [ ]:
from IPython.display import Video
Video(local_file_path)